In [1]:
from bs4 import BeautifulSoup as bs
import requests
from pytube import YouTube
import time

import urllib.request
import urllib.parse
import urllib.error
from bs4 import BeautifulSoup
import ssl
import json
import ast
import json
import os
from urllib.request import Request, urlopen

In [2]:
qstrings = ["red", "green", "blue"]

#initialize request lists
base = "https://www.youtube.com/results?search_query="

rList = []
for i in qstrings:
  rList.append(requests.get(base+i))

soups=[]
for i in rList:
  soups.append(bs(i.text,'html.parser'))

allvids = []
for i in soups:
  allvids.append(i.findAll('a',attrs={'class':'yt-uix-tile-link'}))

flat_vids = []
for sublist in allvids:
    for item in sublist:
        flat_vids.append(item)

# final list of video links here
urllist = []
for v in flat_vids:
  tmp = 'https://www.youtube.com' + v['href']
  urllist.append(tmp)
urllist

['https://www.youtube.com/watch?v=Zlot0i3Zykw',
 'https://www.youtube.com/watch?v=3uD5D2kBpsw',
 'https://www.youtube.com/watch?v=-JZ_moituIo',
 'https://www.youtube.com/watch?v=hMH3nFkvMNo',
 'https://www.youtube.com/watch?v=Ru8LG-_HGd4',
 'https://www.youtube.com/watch?v=CuzkqBSAenk',
 'https://www.youtube.com/watch?v=3vZccD8ySXc',
 'https://www.youtube.com/watch?v=atUwZlW6gEM',
 'https://www.youtube.com/watch?v=gW_LaqKudLY',
 'https://www.youtube.com/watch?v=0TkZTomuTfA',
 'https://www.youtube.com/watch?v=2VbLure1838',
 'https://www.youtube.com/watch?v=lnCr3czDNEs',
 'https://www.youtube.com/watch?v=AgFeZr5ptV8',
 'https://www.youtube.com/watch?v=oKar-tF__ac',
 'https://www.youtube.com/watch?v=wyK7YuwUWsU',
 'https://www.youtube.com/watch?v=nhuXcgMnloo',
 'https://www.youtube.com/watch?v=XG2usu-el68',
 'https://www.youtube.com/watch?v=wGagftdz12Y',
 'https://www.youtube.com/watch?v=3qlRNWBWr5o',
 'https://www.youtube.com/watch?v=rwOJoL2UvzI',
 'https://www.youtube.com/watch?v=8Uw8mI

In [3]:
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

In [4]:
urllist[2]

url = urllist[2]

video_dic = []
id = 1


In [5]:
def fetch_video_details(url,id,description):
    req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
    webpage = urlopen(req).read()
    soup = BeautifulSoup(webpage, 'html.parser')
    html = soup.prettify('utf-8')
#     prefix = "Praty test"
    video_details = {}

    video_details['ID'] = id
    for span in soup.findAll('span',attrs={'class': 'watch-title'}):
        video_details['TITLE'] = span.text.strip()

    for script in soup.findAll('script',attrs={'type': 'application/ld+json'}):
            channelDesctiption = json.loads(script.text.strip())
            video_details['CHANNEL_NAME'] = channelDesctiption['itemListElement'][0]['item']['name']

    for div in soup.findAll('div',attrs={'class': 'watch-view-count'}):
        video_details['NUMBER_OF_VIEWS'] = div.text.strip()

    for button in soup.findAll('button',attrs={'title': 'I like this'}):
        video_details['LIKES'] = button.text.strip()

    for button in soup.findAll('button',attrs={'title': 'I dislike this'}):
        video_details['DISLIKES'] = button.text.strip()

    for span in soup.findAll('span',attrs={'class': 'yt-subscription-button-subscriber-count-branded-horizontal yt-subscriber-count'}):
        video_details['NUMBER_OF_SUBSCRIPTIONS'] = span.text.strip()
    video_details['DESCRIPTION'] = description
    hashtags = []
    # for span in soup.findAll('span',attrs={'class': 'standalone-collection-badge-renderer-text'}):
    #     for a in span.findAll('a',attrs={'class': 'yt-uix-sessionlink'}):
    #         hashtags.append(a.text.strip())
    hashtags = soup.find("meta", {"name":"keywords"})['content']
    video_details['HASH_TAGS'] = hashtags
    
    
    # print(hashtags)
   
    return(video_details)

In [6]:
def download_video(url,prefix):
    urlForYt = url + " -4"
    yt = YouTube(urlForYt)
    videoList = yt.streams.filter(res='360p',mime_type='video/mp4',only_video=True).first()
    print(videoList)
    
    yt.streams.filter(res='360p',mime_type='video/mp4',only_video=True).first().download(filename_prefix=prefix)
    print(url + " downloaded")
    return(yt.description)

In [7]:
id = 1
for i in urllist:
#     fetch_video_details(i)
    prefix=str(id) + "_"
    print("==============downloading video ==========")
    desc = download_video(i,prefix)
    print("==============video details===============")
    
    video_details = fetch_video_details(i, prefix,desc)
#     video_details = video_details[0]
    print(video_details)
    
    id = id + 1

==============downloading video ==========
<Stream: itag="134" mime_type="video/mp4" res="360p" fps="30fps" vcodec="avc1.4d401e">
https://www.youtube.com/watch?v=Zlot0i3Zykw downloaded
==============video details===============
{'CHANNEL_NAME': 'TaylorSwiftVEVO', 'TITLE': 'Taylor Swift - Red', 'HASH_TAGS': 'Taylor Swift, Red, Big Machine, new video, official, live, red tour', 'LIKES': '942,666', 'ID': '1_', 'NUMBER_OF_SUBSCRIPTIONS': '36M', 'DESCRIPTION': 'New single ME! (feat. Brendon Urie of Panic! At The Disco) available now. Download here: https://TaylorSwift.lnk.to/MeYD\n \n►Exclusive Merch: https://store.taylorswift.com\n \n►Follow Taylor Swift Online\nInstagram: http://www.instagram.com/taylorswift\nFacebook: http://www.facebook.com/taylorswift\nTumblr: http://taylorswift.tumblr.com\nTwitter: http://www.twitter.com/taylorswift13\nWebsite: http://www.taylorswift.com\n \n►Follow Taylor Nation Online\nInstagram: http://www.instagram.com/taylornation\nTumblr: http://taylornation.tum

KeyboardInterrupt: 

In [ ]:
def add_info(_url):
  global id
#   yt = YouTube(_url)
  _urlForYt = _url + " -4"
  yt = YouTube(_urlForYt)
  req = Request(_url, headers={'User-Agent': 'Mozilla/5.0'})
  webpage = urlopen(req).read()

  soup = BeautifulSoup(webpage, 'html.parser')
  html = soup.prettify('utf-8')
  prefix = "Praty test"
  try:
    videoList = yt.streams.filter(res='360p',mime_type='video/mp4',only_video=True).first()
    print(videoList)
    yt.streams.filter(res='360p',mime_type='video/mp4',only_video=True).first().download(filename_prefix=prefix)
    print(_url + " downloaded")
    time.sleep(5)

    video_details = {}

    video_details['ID'] = id
    for span in soup.findAll('span',attrs={'class': 'watch-title'}):
        video_details['TITLE'] = span.text.strip()

    for script in soup.findAll('script',attrs={'type': 'application/ld+json'}):
            channelDesctiption = json.loads(script.text.strip())
            video_details['CHANNEL_NAME'] = channelDesctiption['itemListElement'][0]['item']['name']

    for div in soup.findAll('div',attrs={'class': 'watch-view-count'}):
        video_details['NUMBER_OF_VIEWS'] = div.text.strip()

    for button in soup.findAll('button',attrs={'title': 'I like this'}):
        video_details['LIKES'] = button.text.strip()

    for button in soup.findAll('button',attrs={'title': 'I dislike this'}):
        video_details['DISLIKES'] = button.text.strip()

    for span in soup.findAll('span',attrs={'class': 'yt-subscription-button-subscriber-count-branded-horizontal yt-subscriber-count'}):
        video_details['NUMBER_OF_SUBSCRIPTIONS'] = span.text.strip()
    video_details['DESCRIPTION'] = yt.description
    hashtags = []
    # for span in soup.findAll('span',attrs={'class': 'standalone-collection-badge-renderer-text'}):
    #     for a in span.findAll('a',attrs={'class': 'yt-uix-sessionlink'}):
    #         hashtags.append(a.text.strip())
    hashtags = soup.find("meta", {"name":"keywords"})['content']
    video_details['HASH_TAGS'] = hashtags
    id = id + 1
    
    # print(hashtags)
    prefix = str(id) + "_"
    time.sleep(5)
    video_dic.append(video_details)
  except:
    time.sleep(5)
    print (_url + " not reachable")
    pass